In [4]:
import chess
import math
import copy


def static_eval(chess_board):
    # Types:
    """{chess.PAWN, chess.KNIGHT, chess.BISHOP,
             chess.ROOK, chess.QUEEN, chess.KING}"""
    piece_values = [1, 3, 3, 5, 9, 100]
    value = 0
    for i, piece_value in enumerate(piece_values):
        value += len(chess_board.pieces(i+1, True))*piece_value
        value -= len(chess_board.pieces(i+1, False))*piece_value
    return value


def minimax(chess_board, depth, maximizing_player, moves):
    chess_board = copy.deepcopy(chess_board)
    if chess_board.result() == "1-0":
        return ((math.inf, moves) if maximizing_player else (-math.inf, moves))
    elif chess_board.result() == "0-1":
        return ((math.inf), moves if not maximizing_player else (-math.inf, moves))
    elif chess_board.result() == "1/2-1/2" or chess_board.is_stalemate():
        return 0, moves

    if depth == 0:
        return (static_eval(chess_board), moves)

    if maximizing_player:
        max_val = -math.inf
        max_move = None
        for move in chess_board.legal_moves:
            chess_board.push(move)
            value, _ = minimax(chess_board, depth - 1, False, moves + [move])
            chess_board.pop()
            max_val = max(max_val, value)
            if max_val == value:
                max_move = move
        assert chess_board.legal_moves
        assert not max_move == None
        return max_val, moves + [max_move]
    else:
        min_val = +math.inf
        min_move = None
        for move in chess_board.legal_moves:
            chess_board.push(move)
            value, _ = minimax(chess_board, depth - 1, True, moves + [move])

            chess_board.pop()
            min_val = min(min_val, value)
            if min_val == value:
                min_move = move
        assert chess_board.legal_moves
        assert not min_move == None
        return min_val, moves + [min_move]


def minimax_for_color(board, color, depth):
    return minimax(copy.deepcopy(board), depth, color, [])

